In [72]:
# Sattelzeit comparisons

In [73]:
from ipynb.fs.full.koselleck import *

In [74]:
def get_model_paths_sattelzeit():
    df=pd.DataFrame(get_model_paths())
    df['period']=[f'{x}-{y}' for x,y in zip(df.period_start, df.period_end)]
    df=df[df.period.isin({'1700-1770','1770-1830','1830-1900'})]
    return df

In [75]:
dfpaths=get_model_paths_sattelzeit()
dfpaths

Scanning directory for models: 3608it [00:00, 63574.36it/s]


,corpus,period_start,period_end,path,path_vocab,run,period
136,bpo,1700,1770,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_17/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_17/vocab.txt,run_17,1700-1770
137,bpo,1700,1770,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_20/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_20/vocab.txt,run_20,1700-1770
138,bpo,1700,1770,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_24/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_24/vocab.txt,run_24,1700-1770
139,bpo,1700,1770,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_09/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_09/vocab.txt,run_09,1700-1770
140,bpo,1700,1770,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_14/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1700-1770/run_14/vocab.txt,run_14,1700-1770
...,...,...,...,...,...,...,...
1679,bpo,1770,1830,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_19/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_19/vocab.txt,run_19,1770-1830
1680,bpo,1770,1830,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_15/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_15/vocab.txt,run_15,1770-1830
1681,bpo,1770,1830,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_10/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_10/vocab.txt,run_10,1770-1830
1682,bpo,1770,1830,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_02/model.bin,/home/ryan/github/koselleck/data1/models/bpo/1770-1830/run_02/vocab.txt,run_02,1770-1830


In [76]:
dfpaths.period.value_counts()

1770-1830    25
1700-1770    25
1830-1900    12
Name: period, dtype: int64

In [77]:
def do_gen_lnm_sattelzeit(dfrun,k=25,lim=None):
    if not len(dfrun)==2: return pd.DataFrame()
    m1=load_model_row(dfrun.loc['1700-1770'])
    m2=load_model_row(dfrun.loc['1830-1900'])
    
    words=set(m1.wv.key_to_index.keys()) & set(m2.wv.key_to_index.keys()) & set(get_valid_words())
    o=[]
    for w in tqdm(list(words)[:lim],position=0):
        try:
            nb1=[w2 for w2,c in m1.wv.most_similar(w,topn=k*5) if w2 in words][:k]
            nb2=[w2 for w2,c in m2.wv.most_similar(w,topn=k*5) if w2 in words][:k]
            mnb=list(set(nb1)|set(nb2))
            vec1=[m1.wv.similarity(w,w2) for w2 in mnb]
            vec2=[m2.wv.similarity(w,w2) for w2 in mnb]
            lnm=1-fastdist.cosine(vec1,vec2)
            odx={
                'word':w,
                'neighb1':nb1,
                'neighb2':nb2,
                'neighb1_len':len(nb1),
                'neighb2_len':len(nb1),
                'metaneighb_len':len(mnb),
                'lnm':lnm,
            }
        except (KeyError,ValueError) as e:
            continue
        o+=[odx]
    return pd.DataFrame(o)
    

def gen_lnm_sattelzeit(k=25,lim=None,num_proc=1,force=False):
    if not force and os.path.exists(FN_LNM_SATTELZEIT):
        return read_df(FN_LNM_SATTELZEIT)
    dfpaths=get_model_paths_sattelzeit()
    odf=pmap_groups(
        do_gen_lnm_sattelzeit,
        dfpaths.set_index('period').groupby('run'),
        num_proc=num_proc,
        kwargs=dict(k=k,lim=lim)
    )
    odf['lnm_z']=(odf.lnm - odf.lnm.mean())/odf.lnm.std()
    odf['lnm_perc']=odf.lnm.rank() / len(odf.lnm)*100
    odf=odf.sort_values('lnm',ascending=False)
    odf.to_pickle(FN_LNM_SATTELZEIT)
    return odf

In [78]:
# odf=gen_lnm_sattelzeit(num_proc=4,force=False)
# odf

In [79]:
DFCHANGERANK=None
def get_changeranks_sattelzeit(k=25):
    global DFCHANGERANK
    if DFCHANGERANK is not None: return DFCHANGERANK
    odf=gen_lnm_sattelzeit(k=k,num_proc=4,force=False)
    odf=odf.groupby('word').mean().sort_values('lnm',ascending=False)
    odf=odf.query(f'neighb1_len=={k} & neighb2_len=={k}')
    odf['rank_word']=odf.lnm.rank(ascending=False)
    odf['lnm_perc_word']=odf.lnm.rank(ascending=True) / len(odf.lnm) * 100
    DFCHANGERANK=odf
    return odf

In [80]:
# get_changeranks_sattelzeit()

In [81]:
FN_NBR_SATTELZEIT=os.path.join(PATH_DATA,'data.nbr_sattelzeit.pkl')

def do_get_nbr_sattelzeit(wdf,k=25,k2=10):
    w=wdf.iloc[0].word
    nbs1=[(i,x) for nb in wdf.neighb1.iloc[:k2] for i,x in enumerate(nb)]
    nbs2=[(i,x) for nb in wdf.neighb2.iloc[:k2] for i,x in enumerate(nb)]
    counts1=Counter(y for x,y in nbs1)
    counts2=Counter(y for x,y in nbs2)
    ranks1=defaultdict(list)
    ranks2=defaultdict(list)
    for i,x in nbs1: ranks1[x]+=[i]
    for i,x in nbs2: ranks2[x]+=[i]
    ranks1={x:int(np.median(l)) for x,l in ranks1.items()}
    ranks2={x:int(np.median(l)) for x,l in ranks2.items()}

    nbstr1=', '.join(
        f'{"**-" if x not in counts2 else ""}{x}{"**" if x not in counts2 else ""} ({counts2.get(x,0)-counts1[x]:+})'
        for x in sorted(
            list(set(ranks1.keys())&set(counts1.keys())),
#                 key=lambda _: (-counts1[_],ranks1[_])
            key=lambda _: (-(counts1[_]-counts2.get(_,0)),ranks1[_])
        )
    )
    # .{ranks1.get(x,0)-ranks2[x]:+}
    nbstr2=', '.join(
        f'{"**+" if x not in counts1 else ""}{x}{"**" if x not in counts1 else ""} ({counts2[x]-counts1.get(x,0):+})'
        for x in sorted(
            list(set(ranks2.keys())&set(counts2.keys())),
            key=lambda _: (-(counts2[_]-counts1.get(_,0)),ranks2[_])
        )
    )
    #printm(f'| Neighb1 | Neighb2 |\n| -- | -- |\n| {nbstr1} | {nbstr2} |')
    #return
    odx={
        'word':w,
        'neighb1':nbs1,
        'neighb2':nbs2,
        'neighb1_str':nbstr1,
        'neighb2_str':nbstr2,
    }
    return pd.DataFrame([odx])

NBR_STZ=None
def get_nbr_sattelzeit(k=25,k2=10,num_proc=1,force=False):
    global NBR_STZ
    if not force and NBR_STZ is not None: return NBR_STZ
    if not force and os.path.exists(FN_NBR_SATTELZEIT):
        NBR_STZ=read_df(FN_NBR_SATTELZEIT)
        return NBR_STZ
    odf=gen_lnm_sattelzeit(k=k,num_proc=4,force=False)
    odf=odf.query(f'neighb1_len=={k} & neighb2_len=={k}')
    odf=pmap_groups(do_get_nbr_sattelzeit, odf.groupby('word'), num_proc=num_proc, kwargs=dict(k=k,k2=k2))
    odf.to_pickle(FN_NBR_SATTELZEIT)
    NBR_STZ=odf
    return odf

In [82]:
# odf=get_nbr_sattelzeit(num_proc=4)
# odf

In [83]:
def get_nbr_simple_sattelzeit(word_or_words):
    odf=get_nbr_sattelzeit(num_proc=4,force=False).reset_index()
    dfrank=get_changeranks_sattelzeit()
    words=tokenize_fast(word_or_words) if type(word_or_words)==str else word_or_words
    odf=odf[odf.word.isin(set(words))]
    odf=odf.set_index('word').join(dfrank,how='left').sort_values('rank_word')
    odf=odf[['neighb1_str','neighb2_str','rank_word','lnm_perc_word']].sort_values('rank_word')
    odf.neighb1_str=odf.neighb1_str.apply(lambda x: x.replace('** (','</b> (').replace('**','<b>'))
    odf.neighb2_str=odf.neighb2_str.apply(lambda x: x.replace('** (','</b> (').replace('**','<b>'))
    odf.columns=['Neighborhood 1 (Pre-Sattelzeit)', 'Neighborhood 2 (Post-Sattelzeit)','Change Rank', 'Change Percentile']
    return round(odf,0)


In [84]:
# odf=get_nbr_simple_sattelzeit('culture,industry,station,hill,mountain')
# printm(odf.to_markdown())

In [85]:
# odf

## Sattelzeit tilts

In [86]:

def class_change(row,cutoff=.5,sby=col_jjj):
    vec=row.vector.split('.')[0]
    va,vb=vec.split('-') if '-' in vec else (vec,vec)
#         print(vec,va,vb)
    if row[sby]>cutoff:
        return '+'+va
    elif row[sby]<-cutoff:
        return '+'+vb if va!=vb else '-'+va
    else:
        return ''


In [87]:

    
def do_get_tilts_sattelzeit(gdf):
    try:
        gdf=gdf.groupby('period').mean().T
        gdf[col_j]=gdf[col_ii] - gdf[col_i]
        gdf[col_jj]=gdf[col_iii] - gdf[col_ii]
        gdf[col_jjj]=gdf[col_iii] - gdf[col_i]
        gsd=gdf.sort_values(col_jjj,ascending=False)
        gdf=gdf[[col_i, col_j, col_ii, col_jj, col_iii, col_jjj]]
    except KeyError:
        return pd.DataFrame()
    return gdf.rename_axis('vector').reset_index()

DFTILTSTZ=None

def format_change(x,vec):
    posneg=vec.split('.')[0].split('-')
    pos,neg=posneg if len(posneg)==2 else (posneg[0],posneg[0])
    xstr=f'{abs(x):.0f}%'
    pneg=pos if x>0 else neg
    if pos!=neg:
        pstr='+'
    elif x<0:
        pstr='-'
    else:
        pstr='+'
    xstr=f'{pstr}{xstr} {pneg}'
    if abs(x)>=(1 if not percs else 10): xstr=f'<b>{xstr}</b>'
    if pos!=neg: xstr+=f'<br/>{(0.0-x):+.0f}% {pos if pos!=pneg else neg}'
    xstr=f'<i>{xstr}</i>'

    return xstr


def get_tilts_sattelzeit(words=None,fn=FN_VECTOR_SCORES_RUNS_STZ,force=False,num_proc=2,sby='abs(III)',percs=True,**y):
    global DFTILTSTZ
    if not force and DFTILTSTZ is not None: return DFTILTSTZ
    if not force and os.path.exists(fn):
        odf=read_df(fn)
    else:
        odf=compute_key_vector_scores_across_models(
            dfpaths.query('run<="run_10"'),
            words=get_valid_words(),
            num_proc=num_proc,
            kwargs=y
        )
        words=get_valid_words() if not words else words
        odf=odf[odf.word.isin(set(words))]
        odf=pmap_groups(do_get_tilts_sattelzeit, odf.groupby('word'), num_proc=num_proc)
        odf=odf.reset_index().set_index(['word','vector']).sort_index()
        odf['abs(III)']=odf['(III)'].apply(abs)
        odf=odf.sort_values(['word','abs(III)'],ascending=[1,0]).dropna()
        odf.to_pickle(fn)
    
    odf=odf.reset_index()
    odf['change']=odf.apply(class_change,axis=1)
    odf=odf.set_index(['word','vector'])
    
    if percs:
        for ncol in [col_i,col_ii,col_iii]: odf[ncol]=odf[ncol].rank()/len(odf[ncol])*100
        odf[col_j]=odf[col_ii] - odf[col_i]
        odf[col_jj]=odf[col_iii] - odf[col_ii]
        odf[col_jjj]=odf[col_iii] - odf[col_i]
        odf[f'abs{col_jjj}']=odf[col_jjj].apply(abs)

    odf['change_magnitude']=odf[col_jjj].apply(abs)

        
    DFTILTSTZ=odf
    return odf


In [88]:
# %%timeit
odf=get_tilts_sattelzeit(force=False,num_proc=1,percs=True)
odf

1700-1770        (I)  1770-1830       (II)  \
word    vector                                                            
abandon Collective-Indiv.RH  15.625464  12.249344  27.874808  31.447740   
        Polit-Acad.HGI       97.739763  -5.800366  91.939397  -3.259766   
        Woman-Man.VG         26.445140  29.561940  56.007080   1.936550   
        Pos-Neg.HGI          37.810690   4.473437  42.284126  18.527504   
        Abs-Conc.Median      90.022652   0.243229  90.265881   6.988662   
...                                ...        ...        ...        ...   
zoo     Interp-Desc.HGI      17.982250  -9.461182   8.521067  17.834951   
        Woman-Man.VG         44.160643 -18.560306  25.600337  27.278185   
        Judg-Perc.HGI        70.359459  -0.233946  70.125514  -6.850027   
        Abs-Conc.Median      16.220231  -9.482225   6.738006  12.675150   
        Time-Space.HGI       56.765856  -1.755211  55.010645   2.085087   

                             1830-1900      (III)   abs(III)       change  \
word    vector                                                              
abandon Collective-Indiv.RH  59.322548  43.697084  43.697084  +Collective   
        Polit-Acad.HGI       88.679631  -9.060133   9.060133        +Acad   
        Woman-Man.VG         57.943630  31.498490  31.498490       +Woman   
        Pos-Neg.HGI          60.811630  23.000941  23.000941         +Pos   
        Abs-Conc.Median      97.254543   7.231891   7.231891         +Abs   
...                                ...        ...        ...          ...   
zoo     Interp-Desc.HGI      26.356018   8.373768   8.373768                
        Woman-Man.VG         52.878522   8.717879   8.717879                
        Judg-Perc.HGI        63.275486  -7.083973   7.083973                
        Abs-Conc.Median      19.413155   3.192925   3.192925                
        Time-Space.HGI       57.095732   0.329876   0.329876                

                             change_magnitude  
word    vector                                 
abandon Collective-Indiv.RH         43.697084  
        Polit-Acad.HGI               9.060133  
        Woman-Man.VG                31.498490  
        Pos-Neg.HGI                 23.000941  
        Abs-Conc.Median              7.231891  
...                                       ...  
zoo     Interp-Desc.HGI              8.373768  
        Woman-Man.VG                 8.717879  
        Judg-Perc.HGI                7.083973  
        Abs-Conc.Median              3.192925  
        Time-Space.HGI               0.329876  

[161576 rows x 9 columns]

In [89]:
def format_tilts_sattelzeit_html(word,df=None,percs=True):
    if df is None: df=get_tilts_sattelzeit(force=False,num_proc=1,percs=percs).drop(['change','change_magnitude'],1)


    df=df.loc[word]
    def format_change(x,vec):
        posneg=vec.split('.')[0].split('-')
        pos,neg=posneg if len(posneg)==2 else (posneg[0],posneg[0])
#         xstr=f'{x:+.0f}%'
        xstr=f'{abs(x):.0f}%'
        pneg=pos if x>0 else neg
        if pos!=neg:
            pstr='+'
        elif x<0:
            pstr='-'
        else:
            pstr='+'


#         pstr='+' if pneg==pos or pos!=neg else '-'
#         xstr=f'<i>{xstr} ({pstr}{pneg})</i>'
        xstr=f'{pstr}{xstr} {pneg}'
        if abs(x)>=(1 if not percs else 10): xstr=f'<b>{xstr}</b>'
        if pos!=neg: xstr+=f'<br/>{(0.0-x):+.0f}% {pos if pos!=pneg else neg}'
        xstr=f'<i>{xstr}</i>'

#         if x<(-.5 if not percs else -10): xstr=f'''<span style="color:darkred">{xstr}</span>'''
#         if x>(.5 if not percs else 10): xstr=f'''<span style="color:blue">{xstr}</span>'''||
        return xstr
    def format_period(x,vec):
        posneg=vec.split('.')[0].split('-')
        pos,neg=posneg if len(posneg)==2 else (posneg[0],posneg[0])
        pneg=pos if x>(0 if not percs else 50) else neg
        #if pneg==neg and pos!=neg:
        #    x=
        x1=x
        if x<50 and pos!=neg: x=100-x
        xstr=f'{x:.0f}%'
#         xstr=f'{xstr} ({pneg})'
        xstr=f'{xstr} {pneg}'


        if (not percs and abs(x)>=1) or (percs and (x<25 or x>75)):
            xstr=f'<b><u>{xstr}</u></b>'
        if pos!=neg:
            xstr+=f'<br/>{(100-x):.0f}% {pos if pos!=pneg else neg}'

#         if x<(-.5 if not percs else -10): xstr=f'''<span style="color:red">{xstr}</span>'''
#         if x>(.5 if not percs else 10): xstr=f'''<span style="color:blue">{xstr}</span>'''
        return xstr

    df=df.sort_values('abs(III)',ascending=False).drop('abs(III)',1).reset_index()
    for col in df.columns:
        if col in {'vector'}: continue
        if '(I' in col:
            df[col]=[format_change(w,vec) for w,vec in zip(df[col],df.vector)]
        else:
            df[col]=[format_period(w,vec) for w,vec in zip(df[col],df.vector)]
    df['vector']=[f'<u>{x}</u>' for x in df.vector]
#     df=df.set_index('vector')[[col_i,col_ii,col_iii,col_j,col_jj,col_jjj]]
#     df=df.set_index('vector')[[
#         col_i,
#         col_j,
#         col_ii,col_jj,
#         col_iii,col_jjj]]
#     df.columns=[col_j+' '+col_i, 
#                 col_j+'→'+col_jj, 
#                 col_jj+' '+col_ii, 
#                 col_jj+'→'+col_jjj, 
#                 col_jjj+' '+col_iii,
#                 col_j+'→'+col_jjj]
    df=df.set_index('vector')[[
        col_i,col_ii,col_iii,
        col_j,col_jj,col_jjj
    ]]
    
    df.columns=[
        col_j+' '+col_i, 
        col_jj+' '+col_ii, 
        col_jjj+' '+col_iii,
        col_j+'→'+col_jj, 
        col_jj+'→'+col_jjj, 
        col_j+'→'+col_jjj
    ]
        
    return df.to_markdown()

In [90]:

printm(format_tilts_sattelzeit_html('value',percs=True))

| vector                     | (I) 1700-1770                              | (II) 1770-1830                          | (III) 1830-1900                            | (I)→(II)                                     | (II)→(III)                                   | (I)→(III)                                    |
|:---------------------------|:-------------------------------------------|:----------------------------------------|:-------------------------------------------|:---------------------------------------------|:---------------------------------------------|:---------------------------------------------|
| <u>Collective-Indiv.RH</u> | <b><u>75% Indiv</u></b><br/>25% Collective | 53% Collective<br/>47% Indiv            | <b><u>85% Collective</u></b><br/>15% Indiv | <i><b>+28% Collective</b><br/>-28% Indiv</i> | <i><b>+33% Collective</b><br/>-33% Indiv</i> | <i><b>+61% Collective</b><br/>-61% Indiv</i> |
| <u>Polit-Acad.HGI</u>      | 60% Polit<br/>40% Acad                     | 56% Acad<br/>44% Polit                  | <b><u>91% Acad</u></b><br/>9% Polit        | <i><b>+17% Acad</b><br/>+17% Polit</i>       | <i><b>+35% Acad</b><br/>+35% Polit</i>       | <i><b>+52% Acad</b><br/>+52% Polit</i>       |
| <u>Interp-Desc.HGI</u>     | 59% Interp<br/>41% Desc                    | 65% Interp<br/>35% Desc                 | <b><u>92% Interp</u></b><br/>8% Desc       | <i>+6% Interp<br/>-6% Desc</i>               | <i><b>+27% Interp</b><br/>-27% Desc</i>      | <i><b>+33% Interp</b><br/>-33% Desc</i>      |
| <u>Abs-Conc.Median</u>     | 53% Abs<br/>47% Conc                       | 61% Abs<br/>39% Conc                    | <b><u>81% Abs</u></b><br/>19% Conc         | <i>+8% Abs<br/>-8% Conc</i>                  | <i><b>+21% Abs</b><br/>-21% Conc</i>         | <i><b>+28% Abs</b><br/>-28% Conc</i>         |
| <u>Pos-Neg.HGI</u>         | 74% Pos<br/>26% Neg                        | <b><u>82% Pos</u></b><br/>18% Neg       | <b><u>98% Pos</u></b><br/>2% Neg           | <i>+8% Pos<br/>-8% Neg</i>                   | <i><b>+16% Pos</b><br/>-16% Neg</i>          | <i><b>+24% Pos</b><br/>-24% Neg</i>          |
| <u>Time-Space.HGI</u>      | 69% Space<br/>31% Time                     | <b><u>76% Space</u></b><br/>24% Time    | 53% Time<br/>47% Space                     | <i>+7% Space<br/>+7% Time</i>                | <i><b>+29% Time</b><br/>-29% Space</i>       | <i><b>+22% Time</b><br/>-22% Space</i>       |
| <u>Strong-Weak.HGI</u>     | 73% Strong<br/>27% Weak                    | <b><u>80% Strong</u></b><br/>20% Weak   | <b><u>94% Strong</u></b><br/>6% Weak       | <i>+8% Strong<br/>-8% Weak</i>               | <i><b>+14% Strong</b><br/>-14% Weak</i>      | <i><b>+22% Strong</b><br/>-22% Weak</i>      |
| <u>Virtue-Vice.HGI</u>     | <b><u>80% Virtue</u></b><br/>20% Vice      | <b><u>88% Virtue</u></b><br/>12% Vice   | <b><u>99% Virtue</u></b><br/>1% Vice       | <i>+7% Virtue<br/>-7% Vice</i>               | <i><b>+11% Virtue</b><br/>-11% Vice</i>      | <i><b>+18% Virtue</b><br/>-18% Vice</i>      |
| <u>Ambig.NX</u>            | 70% Ambig                                  | <b><u>19% Ambig</u></b>                 | <b><u>83% Ambig</u></b>                    | <i><b>-50% Ambig</b></i>                     | <i><b>+64% Ambig</b></i>                     | <i><b>+13% Ambig</b></i>                     |
| <u>Pleasure-Pain.HGI</u>   | <b><u>82% Pleasure</u></b><br/>18% Pain    | <b><u>81% Pleasure</u></b><br/>19% Pain | <b><u>91% Pleasure</u></b><br/>9% Pain     | <i>+0% Pain<br/>+0% Pleasure</i>             | <i>+10% Pleasure<br/>-10% Pain</i>           | <i>+9% Pleasure<br/>-9% Pain</i>             |
| <u>Woman-Man.VG</u>        | 74% Man<br/>26% Woman                      | 55% Man<br/>45% Woman                   | 66% Man<br/>34% Woman                      | <i><b>+19% Woman</b><br/>-19% Man</i>        | <i><b>+11% Man</b><br/>+11% Woman</i>        | <i>+7% Woman<br/>-7% Man</i>                 |
| <u>Sing-Plural.M</u>       | <b><u>93% Sing</u></b><br/>7% Plural       | <b><u>87% Sing</u></b><br/>13% Plural   | <b><u>100% Sing</u></b><br/>0% Plural      | <i>+6% Plural<br/>+6% Sing</i>               | <i><b>+13% Sing</b><br/>-13% Plural</i>      | <i>+7% Sing<br/>-7% Plural</i>               |
| <u>Judg-Perc.HGI</u>       | <b><u>82% Perc</u></b><br/>18% Judg        | <b><u>84% Perc</u></b><br/>16% Judg     | <b><u>80% Perc</u></b><br/>20% Judg        | <i>+2% Perc<br/>+2% Judg</i>                 | <i>+5% Judg<br/>-5% Perc</i>                 | <i>+3% Judg<br/>-3% Perc</i>                 |
| <u>Human-Object.VG</u>     | 70% Object<br/>30% Human                   | 73% Object<br/>27% Human                | 68% Object<br/>32% Human                   | <i>+3% Object<br/>+3% Human</i>              | <i>+5% Human<br/>-5% Object</i>              | <i>+2% Human<br/>-2% Object</i>              |
| <u>Active-Passive.HGI</u>  | 60% Active<br/>40% Passive                 | 59% Active<br/>41% Passive              | 58% Active<br/>42% Passive                 | <i>+1% Passive<br/>+1% Active</i>            | <i>+0% Passive<br/>+0% Active</i>            | <i>+2% Passive<br/>+2% Active</i>            |
| <u>Freq.M</u>              | <b><u>99% Freq</u></b>                     | <b><u>98% Freq</u></b>                  | <b><u>100% Freq</u></b>                    | <i>-0% Freq</i>                              | <i>+2% Freq</i>                              | <i>+1% Freq</i>                              |
| <u>Qual-Quant.HGI</u>      | <b><u>99% Quant</u></b><br/>1% Qual        | <b><u>100% Quant</u></b><br/>0% Qual    | <b><u>99% Quant</u></b><br/>1% Qual        | <i>+0% Quant<br/>+0% Qual</i>                | <i>+1% Qual<br/>-1% Quant</i>                | <i>+0% Qual<br/>-0% Quant</i>                |

In [91]:
def get_tilts_sattelzeit_word(w):
    df=get_tilts_sattelzeit().loc[w]
    df=df[[
        col_i,col_ii,col_iii,
        col_j,col_jj,col_jjj,
        'change','change_magnitude'
    ]]
    scol='change_magnitude'
    df=df.sort_values(scol,ascending=False)
    
    df.columns=[
        col_j+' '+col_i, 
        col_jj+' '+col_ii, 
        col_jjj+' '+col_iii,
        col_j+'→'+col_jj, 
        col_jj+'→'+col_jjj, 
        col_j+'→'+col_jjj,
        'Change Direction','Change Magnitude'
    ]
    return df.sort_values(df.columns[-1], ascending=False)


In [92]:
round(get_tilts_sattelzeit_word('culture'),1)

,(I) 1700-1770,(II) 1770-1830,(III) 1830-1900,(I)→(II),(II)→(III),(I)→(III),Change Direction,Change Magnitude
vector,,,,,,,,
Ambig.NX,31.4,42.6,77.9,11.2,35.3,46.5,+Ambig,46.5
Human-Object.VG,14.9,12.4,59.9,-2.5,47.5,45.0,+Human,45.0
Abs-Conc.Median,46.9,53.5,88.7,6.5,35.2,41.7,+Abs,41.7
Active-Passive.HGI,62.9,62.5,39.1,-0.4,-23.4,-23.8,+Passive,23.8
Qual-Quant.HGI,43.9,25.7,22.5,-18.2,-3.2,-21.4,+Quant,21.4
Interp-Desc.HGI,75.4,77.8,95.3,2.4,17.5,20.0,+Interp,20.0
Freq.M,41.0,43.8,57.5,2.8,13.7,16.5,,16.5
Strong-Weak.HGI,82.3,83.3,96.1,1.0,12.8,13.8,+Strong,13.8
Pos-Neg.HGI,94.7,91.9,99.6,-2.9,7.7,4.9,+Pos,4.9


In [93]:

def get_top_changes_stz(n=200,sby='(III)'):
    odf=get_tilts_sattelzeit().groupby(['vector','word']).mean()
    odf=odf.sort_values(['vector',sby],ascending=[1,0])
    od={}
    for vec,vecdf in odf.groupby('vector'):
        posneg=vec.split('.')[0].split('-')
        pos,neg=posneg if len(posneg)==2 else (posneg[0],posneg[0])
        od[f'+{pos}']=vecdf.sort_values(sby,ascending=False).head(n).index.get_level_values('word')
        od[f'+{neg}' if pos!=neg else f'-{neg}']=vecdf.sort_values(sby,ascending=True).head(n).index.get_level_values('word')
    return od

In [94]:
get_top_changes_stz()

{'+Abs': Index(['lack', 'lax', 'tact', 'lucid', 'beset', 'prolific', 'frenchmen',
        'role', 'deals', 'viewed',
        ...
        'alive', 'weighed', 'reviewing', 'occupy', 'unavailing', 'confided',
        'sweetest', 'rates', 'finale', 'finer'],
       dtype='object', name='word', length=200),
 '+Conc': Index(['muff', 'mull', 'slate', 'hewn', 'conduit', 'slates', 'stile',
        'surmounted', 'frill', 'pleated',
        ...
        'correspond', 'shining', 'ornaments', 'quaint', 'companion', 'inserted',
        'moving', 'dress', 'crept', 'sovereigns'],
       dtype='object', name='word', length=200),
 '+Active': Index(['completion', 'filly', 'flue', 'temperance', 'mull', 'moderate',
        'slate', 'local', 'fees', 'fins',
        ...
        'inventions', 'miscellany', 'advertiser', 'invites', 'fake', 'afford',
        'founding', 'opponent', 'termed', 'objects'],
       dtype='object', name='word', length=200),
 '+Passive': Index(['confiding', 'trance', 'touching', 'veile

In [95]:
get_top_changes_stz()['-Ambig']

Index(['lung', 'repute', 'firs', 'sue', 'assurances', 'interposed', 'baring',
       'ides', 'intent', 'vest',
       ...
       'generation', 'barter', 'hug', 'tribe', 'table', 'ended', 'colours',
       'detestable', 'hat', 'curl'],
      dtype='object', name='word', length=200)

In [101]:
def get_topmost_change(sby='(III)',cutoff=.5):
    sdf=get_tilts_sattelzeit().sort_values(['word','abs(III)'],ascending=[1,0])
    sdf1=sdf.groupby('word').head(1).reset_index()
    def class_change(row):
        vec=row.vector.split('.')[0]
        va,vb=vec.split('-') if '-' in vec else (vec,vec)
#         print(vec,va,vb)
        if row[sby]>cutoff:
            return '+'+va
        elif row[sby]<-cutoff:
            return '+'+vb if va!=vb else '-'+va
        else:
            return ''

    sdf1['change']=sdf1.apply(class_change,axis=1)
    sdf1=sdf1.set_index('word')
    return sdf1

DFALLSGNCH=None
def get_all_signif_changes(cutoff=.5,sby='(III)'):
    global DFALLSGNCH
    if DFALLSGNCH is None:
        sdf=get_tilts_sattelzeit().sort_values(['word','abs(III)'],ascending=[1,0])
        sdf1=sdf.reset_index()
        sdf1['change']=sdf1.apply(class_change,axis=1)
        sdf1=sdf1.set_index('word')
        DFALLSGNCH=sdf1[sdf1.change!='']
    return DFALLSGNCH


In [ ]:
get_all_signif_changes()